In [ ]:
import pandas as pd
import numpy as np
import utils 
df = pd.read_parquet('/dat/yuwang/project/data.parquet')
train_data = df.query('datetime < "2021-01-01"')
test_data = df.query('datetime >= "2021-01-01"')
mtc = utils.ModelMetrics(test_data.benchmark_yhat,test_data.y_1,test_data.datetime.apply(lambda x: x.split(' ')[0]))

In [ ]:
import utils
from models.common import Pipeline
from models.dnn import    Net
import pytorch_lightning as pl
import wandb
import os
from datetime import datetime

os.environ["CUDA_VISIBLE_DEVICES"] = "1,3"


timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M")
# wandb.init(entity="sfcap", project="ml")
RET_SCALER = 1
BATCH_SIZE = 512

X_train = utils.df_train.filter(regex="X_")
X_test = utils.df_test.filter(regex="X_")
y_train = utils.df_train["y_2"] * RET_SCALER
y_test = utils.df_test["y_1"] * RET_SCALER


model = Net(
    input_size=260,
    hidden_sizes=[200, 100, 50],
    output_size=1,
    lr=1e-3,
    act="SELU",
    loss_fn="mse",
)
datamodule = utils.FinanceDataModule(
    X_train, X_test, y_train, y_test, batch_size=BATCH_SIZE
)


In [ ]:

trainer = Pipeline(early_stop_patient=3, experiment_name=timestamp + "_y2",devices=1)

In [ ]:
trainer.make(model, datamodule, 1)
# trainer.evaluate( datamodule, utils.test_date)


In [ ]:
datamodule.test_dataloader()

In [ ]:
tmp = trainer.trainer

In [ ]:
import pandas as pd

In [39]:
df = pd.read_parquet('/dat/yuwang/project/data.parquet')

In [40]:
grouped_data = df.groupby('datetime')

# 创建一个空的DataFrame，用于存储结果
ranked_data = pd.DataFrame()

# 对每个分组计算横截面排序
for datetime, group in grouped_data:
    group = group.copy()  # 复制以避免SettingWithCopyWarning
    group['rank'] = group['y_1'].rank(ascending=False)
    ranked_data = ranked_data.append(group)

# 按照原始索引重新排序结果
ranked_data = ranked_data.sort_index()

/tmp/ipykernel_146173/1364551777.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_data = ranked_data.append(group)
/tmp/ipykernel_146173/1364551777.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_data = ranked_data.append(group)
/tmp/ipykernel_146173/1364551777.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_data = ranked_data.append(group)
/tmp/ipykernel_146173/1364551777.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ranked_data = ranked_data.append(group)
/tmp/ipykernel_146173/1364551777.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ver

In [46]:
ranked_data['rank'].describe()

count    7.608924e+06
mean     1.797844e+03
std      1.044893e+03
min      1.000000e+00
25%      8.960000e+02
50%      1.791500e+03
75%      2.687000e+03
max      3.917000e+03
Name: rank, dtype: float64

In [53]:
def mark_rank_label(rank):
    if rank < 8.960000e+02:
        return 1
    elif rank <  1.791500e+03:
        return 2
    elif rank < 2.687000e+03:
        return 3
    else:
        return 4

In [54]:
ranked_data['rank_label'] = ranked_data.apply(lambda x: mark_rank_label(x['rank']),axis=1)

In [55]:
ranked_data.to_parquet('/dat/yuwang/project/data_rank.parquet')